In [ ]:
# for similarity need to download larger english word library with the below in gitbash 
# python -m spacy download en_core_web_lg

In [1]:
import numpy as np
import pandas as pd
import spacy
import pyspark
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

In [3]:
file = pd.read_csv("final_moviedata.csv")
file_all = pd.read_csv("final_moviedata.csv")

In [4]:
file = pd.DataFrame(file)
file.head()

,imdb_id,title,original_title,overview,poster_link
0,tt0069049,The Other Side of the Wind,The Other Side of the Wind,"Orson Welles' unfinished masterpiece, restored...",https://m.media-amazon.com/images/M/MV5BNDg5NT...
1,tt3906082,Mary Shelley,Mary Shelley,The love affair between poet Percy Shelley and...,https://m.media-amazon.com/images/M/MV5BNzUyM2...
2,tt5613402,Mobile Homes,Mobile Homes,"In forgotten towns along the American border, ...",https://m.media-amazon.com/images/M/MV5BMTc3OT...
3,tt3038708,Iron Sky: The Coming Race,Iron Sky: The Coming Race,"Twenty years after the events of Iron Sky, the...",https://m.media-amazon.com/images/M/MV5BYzFmZD...
4,tt3498786,Sly Cooper,Sly Cooper,Sly Cooper is a kinetic and comedic heist film...,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...


In [5]:
file_all = pd.DataFrame(file_all)
file_all.head()

,imdb_id,title,original_title,overview,poster_link
0,tt0069049,The Other Side of the Wind,The Other Side of the Wind,"Orson Welles' unfinished masterpiece, restored...",https://m.media-amazon.com/images/M/MV5BNDg5NT...
1,tt3906082,Mary Shelley,Mary Shelley,The love affair between poet Percy Shelley and...,https://m.media-amazon.com/images/M/MV5BNzUyM2...
2,tt5613402,Mobile Homes,Mobile Homes,"In forgotten towns along the American border, ...",https://m.media-amazon.com/images/M/MV5BMTc3OT...
3,tt3038708,Iron Sky: The Coming Race,Iron Sky: The Coming Race,"Twenty years after the events of Iron Sky, the...",https://m.media-amazon.com/images/M/MV5BYzFmZD...
4,tt3498786,Sly Cooper,Sly Cooper,Sly Cooper is a kinetic and comedic heist film...,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...


In [6]:
file_all.count()

imdb_id           1886
title             1886
original_title    1886
overview          1886
poster_link       1886
dtype: int64

In [7]:
file.drop_duplicates(keep='first', inplace=True)
file_all.drop_duplicates(keep='first', inplace=True)

In [8]:
file.count()

imdb_id           1886
title             1886
original_title    1886
overview          1886
poster_link       1886
dtype: int64

In [9]:
overview_df2 = file[["imdb_id","title", "poster_link", "overview"]].copy()
overview_df2.dropna(inplace=True)
overview_df2 = overview_df2.iloc[137:200]
overview_df2.head()

,imdb_id,title,poster_link,overview
137,tt5776208,Resident Evil: Vendetta,https://m.media-amazon.com/images/M/MV5BMTg0OT...,BSAA Chris Redfield enlists the help of govern...
138,tt6083388,God of War,https://m.media-amazon.com/images/M/MV5BZmQ5Mj...,"During the 16th century, Japanese pirates prol..."
139,tt4758646,War Machine,https://m.media-amazon.com/images/M/MV5BMTcyMz...,A rock star general bent on winning the ‚impos...
140,tt6367558,The Double Lover,https://m.media-amazon.com/images/M/MV5BYzdhMz...,"Chlov, a fragile young woman, falls in love wi..."
141,tt5278598,Kiki,https://ia.media-imdb.com/images/M/MV5BMTZiOGF...,"25 years after Paris is Burning, we dive back ..."


In [10]:
overview_df2. count()

imdb_id        63
title          63
poster_link    63
overview       63
dtype: int64

In [11]:
df_all = file_all[["imdb_id","title", "poster_link", "overview"]].copy()
df_all.dropna(inplace=True)
df_all.head()

,imdb_id,title,poster_link,overview
0,tt0069049,The Other Side of the Wind,https://m.media-amazon.com/images/M/MV5BNDg5NT...,"Orson Welles' unfinished masterpiece, restored..."
1,tt3906082,Mary Shelley,https://m.media-amazon.com/images/M/MV5BNzUyM2...,The love affair between poet Percy Shelley and...
2,tt5613402,Mobile Homes,https://m.media-amazon.com/images/M/MV5BMTc3OT...,"In forgotten towns along the American border, ..."
3,tt3038708,Iron Sky: The Coming Race,https://m.media-amazon.com/images/M/MV5BYzFmZD...,"Twenty years after the events of Iron Sky, the..."
4,tt3498786,Sly Cooper,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...,Sly Cooper is a kinetic and comedic heist film...


In [12]:
overview_df2.isna().any()

imdb_id        False
title          False
poster_link    False
overview       False
dtype: bool

In [13]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [14]:
sentenceData = spark.createDataFrame(overview_df2.astype(str))
sentenceData.show() 

+---------+--------------------+--------------------+--------------------+
|  imdb_id|               title|         poster_link|            overview|
+---------+--------------------+--------------------+--------------------+
|tt5776208|Resident Evil: Ve...|https://m.media-a...|BSAA Chris Redfie...|
|tt6083388|          God of War|https://m.media-a...|During the 16th c...|
|tt4758646|         War Machine|https://m.media-a...|A rock star gener...|
|tt6367558|    The Double Lover|https://m.media-a...|Chlov, a fragile ...|
|tt5278598|                Kiki|https://ia.media-...|25 years after Pa...|
|tt5352846|               Drone|https://m.media-a...|Ideologies collid...|
|tt2090465|     Black Butterfly|https://m.media-a...|Paul is a down-on...|
|tt5031332|             McLaren|https://m.media-a...|The story of New ...|
|tt2674454|           Churchill|https://m.media-a...|A ticking-clock t...|
|tt3335606|     Berlin Syndrome|https://m.media-a...|A passionate holi...|
|tt5262792|       Dirty D

In [15]:
sentenceData_all = spark.createDataFrame(df_all.astype(str))
sentenceData_all.show() 

+---------+--------------------+--------------------+--------------------+
|  imdb_id|               title|         poster_link|            overview|
+---------+--------------------+--------------------+--------------------+
|tt0069049|The Other Side of...|https://m.media-a...|Orson Welles' unf...|
|tt3906082|        Mary Shelley|https://m.media-a...|The love affair b...|
|tt5613402|        Mobile Homes|https://m.media-a...|In forgotten town...|
|tt3038708|Iron Sky: The Com...|https://m.media-a...|Twenty years afte...|
|tt3498786|          Sly Cooper|http://ia.media-i...|Sly Cooper is a k...|
|tt2328678| The King's Daughter|https://m.media-a...|King Louis XIV's ...|
|tt3859310|How to Talk to Gi...|https://m.media-a...|A couple of Briti...|
|tt4765284|     Pitch Perfect 3|https://m.media-a...|Sequel to Pitch P...|
|tt5690244|            Machines|https://m.media-a...|This portrayal of...|
|tt2582498|      Sweet Virginia|https://m.media-a...|A former rodeo ch...|
|tt0974015|      Justice 

In [16]:
tokenizer = Tokenizer(inputCol="overview", outputCol="words")
tokenizer

Tokenizer_ad164150d005

In [17]:
# Transform and show DataFrame
tokenized = tokenizer.transform(sentenceData)
tokenized_all = tokenizer.transform(sentenceData_all)
tokenized.show(truncate=False)

+---------+------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="words", outputCol="nostopwords")

In [19]:
# Transform and show data
tokenized = remover.transform(tokenized)
tokenized_all = remover.transform(tokenized_all)
tokenized.show(truncate=True)

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  imdb_id|               title|         poster_link|            overview|               words|         nostopwords|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|tt5776208|Resident Evil: Ve...|https://m.media-a...|BSAA Chris Redfie...|[bsaa, chris, red...|[bsaa, chris, red...|
|tt6083388|          God of War|https://m.media-a...|During the 16th c...|[during, the, 16t...|[16th, century,, ...|
|tt4758646|         War Machine|https://m.media-a...|A rock star gener...|[a, rock, star, g...|[rock, star, gene...|
|tt6367558|    The Double Lover|https://m.media-a...|Chlov, a fragile ...|[chlov,, a, fragi...|[chlov,, fragile,...|
|tt5278598|                Kiki|https://ia.media-...|25 years after Pa...|[25, years, after...|[25, years, paris...|
|tt5352846|               Drone|https://m.media-a...|Ideologies 

In [20]:
tokenized_all.show(truncate=True)

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  imdb_id|               title|         poster_link|            overview|               words|         nostopwords|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|tt0069049|The Other Side of...|https://m.media-a...|Orson Welles' unf...|[orson, welles', ...|[orson, welles', ...|
|tt3906082|        Mary Shelley|https://m.media-a...|The love affair b...|[the, love, affai...|[love, affair, po...|
|tt5613402|        Mobile Homes|https://m.media-a...|In forgotten town...|[in, forgotten, t...|[forgotten, towns...|
|tt3038708|Iron Sky: The Com...|https://m.media-a...|Twenty years afte...|[twenty, years, a...|[twenty, years, e...|
|tt3498786|          Sly Cooper|http://ia.media-i...|Sly Cooper is a k...|[sly, cooper, is,...|[sly, cooper, kin...|
|tt2328678| The King's Daughter|https://m.media-a...|King Louis 

# Starting Spacy Similarity code

In [21]:
# Load English tokenizer, tagger, parser, NER and word vectors
#MAKE SURE THIS IS LARGE LIBRARY

nlp = spacy.load("en_core_web_lg")



## adding columns for similarities

In [22]:
from pyspark.sql.functions import lit

In [23]:
#Code to compare each movie to the all movie and recommend top 3
ll = []
result = []
for m in tokenized.collect():
    main_row = str(m.nostopwords)
    docA = nlp(main_row)
    for f in tokenized_all.collect():

        overview_row = str(f.nostopwords)
        #print(overview_row)
        doc_row = nlp(overview_row)

        compare = docA.similarity(doc_row)
        ll.append({'imdb_id': f.imdb_id, 'Title': f.title,'Poster': f.poster_link,
           'Similarity': compare })
        # sample = tokenized.withColumn('similarity', lit(str(ll)))
        # print(ll)   
        df_simi = pd.DataFrame(ll)
        df_simi.sort_values(by=['Similarity'], inplace=True, ascending=False)
        df_simi.reset_index(drop=True)
        top3_df = df_simi.iloc[1:4]
        top3_df = top3_df[["imdb_id","Title", "Poster", "Similarity"]]
#         top3_df.head()
    result.append({'imdb_id': m.imdb_id,'Title': m.title,'Poster': m.poster_link, 
        'Recom1': top3_df.iloc[0,0], 'Recom1Poster': top3_df.iloc[0,2],
        'Recom2': top3_df.iloc[1,0], 'Recom2Poster': top3_df.iloc[1,2],
        'Recom3': top3_df.iloc[2,0],'Recom3Poster': top3_df.iloc[2,2]})
    print(result)


[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}]
[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 

[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}, {'imdb_id': 'tt6083388', 'Title': 'God of War', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Recom1': 'tt6083388', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Reco

[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}, {'imdb_id': 'tt6083388', 'Title': 'God of War', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Recom1': 'tt6083388', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Reco

[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}, {'imdb_id': 'tt6083388', 'Title': 'God of War', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Recom1': 'tt6083388', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Reco

[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}, {'imdb_id': 'tt6083388', 'Title': 'God of War', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Recom1': 'tt6083388', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Reco

...

[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}, {'imdb_id': 'tt6083388', 'Title': 'God of War', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Recom1': 'tt6083388', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Reco

[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}, {'imdb_id': 'tt6083388', 'Title': 'God of War', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Recom1': 'tt6083388', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Reco

[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}, {'imdb_id': 'tt6083388', 'Title': 'God of War', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Recom1': 'tt6083388', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Reco

[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}, {'imdb_id': 'tt6083388', 'Title': 'God of War', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Recom1': 'tt6083388', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Reco

[{'imdb_id': 'tt5776208', 'Title': 'Resident Evil: Vendetta', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0OTQ0ODY1N15BMl5BanBnXkFtZTgwNjkxNjI4MTI@._V1_SX300.jpg', 'Recom1': 'tt5952526', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNDIzNWUyNzMtMGFiNy00MjIyLTk3ZjItZTRjNDM1MDNlNmQ2XkEyXkFqcGdeQXVyNDgxMDU4NTU@._V1_SX300.jpg', 'Recom2': 'tt3014866', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTg0ODc4Mzk2OF5BMl5BanBnXkFtZTgwNDk2MDkyODE@._V1_SX300.jpg', 'Recom3': 'tt3685622', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BMjEzNTgxMDQ0NF5BMl5BanBnXkFtZTgwNzM2NzkyNzE@._V1_SX300.jpg'}, {'imdb_id': 'tt6083388', 'Title': 'God of War', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Recom1': 'tt6083388', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BZmQ5MjE4NDgtZjAxNC00MDVhLTlkNWQtZDNlYTU5NTA5NTM3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_SX300.jpg', 'Reco

In [ ]:
result

# Importing recommandation to csv

In [24]:
df_recomm = pd.DataFrame(result)
df_recomm.drop_duplicates(keep='first', inplace=True)
df_recomm.head()

,Poster,Recom1,Recom1Poster,Recom2,Recom2Poster,Recom3,Recom3Poster,Title,imdb_id
0,https://m.media-amazon.com/images/M/MV5BMTg0OT...,tt5952526,https://m.media-amazon.com/images/M/MV5BNDIzNW...,tt3014866,https://m.media-amazon.com/images/M/MV5BMTg0OD...,tt3685622,https://m.media-amazon.com/images/M/MV5BMjEzNT...,Resident Evil: Vendetta,tt5776208
1,https://m.media-amazon.com/images/M/MV5BZmQ5Mj...,tt6083388,https://m.media-amazon.com/images/M/MV5BZmQ5Mj...,tt6078822,https://m.media-amazon.com/images/M/MV5BZmVhM2...,tt5282006,https://m.media-amazon.com/images/M/MV5BMjA0MD...,God of War,tt6083388
2,https://m.media-amazon.com/images/M/MV5BMTcyMz...,tt4758646,https://m.media-amazon.com/images/M/MV5BMTcyMz...,tt6083388,https://m.media-amazon.com/images/M/MV5BZmQ5Mj...,tt6078822,https://m.media-amazon.com/images/M/MV5BZmVhM2...,War Machine,tt4758646
3,https://m.media-amazon.com/images/M/MV5BYzdhMz...,tt5776208,https://m.media-amazon.com/images/M/MV5BMTg0OT...,tt4758646,https://m.media-amazon.com/images/M/MV5BMTcyMz...,tt6367558,https://m.media-amazon.com/images/M/MV5BYzdhMz...,The Double Lover,tt6367558
4,https://ia.media-imdb.com/images/M/MV5BMTZiOGF...,tt5776208,https://m.media-amazon.com/images/M/MV5BMTg0OT...,tt6083388,https://m.media-amazon.com/images/M/MV5BZmQ5Mj...,tt4758646,https://m.media-amazon.com/images/M/MV5BMTcyMz...,Kiki,tt5278598


In [25]:
#Reorganize the column
df_recomm = df_recomm[["imdb_id","Title", "Poster", 
        "Recom1", "Recom1Poster", "Recom2", "Recom2Poster",
                      "Recom3", "Recom3Poster"]]
df_recomm.head()

,imdb_id,Title,Poster,Recom1,Recom1Poster,Recom2,Recom2Poster,Recom3,Recom3Poster
0,tt5776208,Resident Evil: Vendetta,https://m.media-amazon.com/images/M/MV5BMTg0OT...,tt5952526,https://m.media-amazon.com/images/M/MV5BNDIzNW...,tt3014866,https://m.media-amazon.com/images/M/MV5BMTg0OD...,tt3685622,https://m.media-amazon.com/images/M/MV5BMjEzNT...
1,tt6083388,God of War,https://m.media-amazon.com/images/M/MV5BZmQ5Mj...,tt6083388,https://m.media-amazon.com/images/M/MV5BZmQ5Mj...,tt6078822,https://m.media-amazon.com/images/M/MV5BZmVhM2...,tt5282006,https://m.media-amazon.com/images/M/MV5BMjA0MD...
2,tt4758646,War Machine,https://m.media-amazon.com/images/M/MV5BMTcyMz...,tt4758646,https://m.media-amazon.com/images/M/MV5BMTcyMz...,tt6083388,https://m.media-amazon.com/images/M/MV5BZmQ5Mj...,tt6078822,https://m.media-amazon.com/images/M/MV5BZmVhM2...
3,tt6367558,The Double Lover,https://m.media-amazon.com/images/M/MV5BYzdhMz...,tt5776208,https://m.media-amazon.com/images/M/MV5BMTg0OT...,tt4758646,https://m.media-amazon.com/images/M/MV5BMTcyMz...,tt6367558,https://m.media-amazon.com/images/M/MV5BYzdhMz...
4,tt5278598,Kiki,https://ia.media-imdb.com/images/M/MV5BMTZiOGF...,tt5776208,https://m.media-amazon.com/images/M/MV5BMTg0OT...,tt6083388,https://m.media-amazon.com/images/M/MV5BZmQ5Mj...,tt4758646,https://m.media-amazon.com/images/M/MV5BMTcyMz...


In [ ]:
#Use this code for the first time to create the csv file
# df_recomm.to_csv('/Users/ericH/Desktop/moviesDashboard/raw/recom.csv', 
#                  index = None, header=True)


In [ ]:
#Use this code to add to csv file
# df_recomm.to_csv('/Users/ericH/Desktop/moviesDashboard/raw/recom.csv', mode='a', index = None, header=False)


In [26]:
file_path = '/Users/ericH/Desktop/moviesDashboard/raw/recom.csv'
with open(file_path, mode='a', newline='\n') as f:
            df_recomm.to_csv(f, index = None, header=False)